# Random Forest Regression on HMM Sleep Data 

In [ ]:
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split, GroupKFold
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import check_random_state
from sklearn.model_selection import GridSearchCV, GroupKFold
from math import sqrt


## Functions
**Create scorer's for the models to assess their fit**

In [ ]:
# Custom RMSE scorer
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Custom calibration slope scorer
def calibration_slope(y_true, y_pred):
    model = LinearRegression()
    model.fit(y_pred.reshape(-1, 1), y_true)
    return model.coef_[0]

# Custom calibration intercept scorer
def calibration_intercept(y_true, y_pred):
    model = LinearRegression()
    model.fit(y_pred.reshape(-1, 1), y_true)
    return model.intercept_

# Define the scorer's dictionary
scorers = {
    'rmse': make_scorer(rmse),
    'r2': make_scorer(r2_score),
    'mae': make_scorer(mean_absolute_error),
    'explained_variance': make_scorer(explained_variance_score),
    'calibration_slope': make_scorer(calibration_slope),
    'calibration_intercept': make_scorer(calibration_intercept)
}

## Dataset up 

In [ ]:
df_sleep = pd.read_csv("Sleep_total_data_scaled_probs.csv")
df_activity = pd.read_csv("Act_total_data_scaled_probs.csv")
df_predictors = pd.read_csv("df_predictors.csv")

In [ ]:
# SLEEP!!!! 
df_sleep = df_sleep.rename(columns={'hmm': 'HMM_State_Sleep', 
                                    'State_Prob_1': 'State_Prob_1_Sleep',
                                    'State_Prob_2': 'State_Prob_2_Sleep',
                                    'State_Prob_3': 'State_Prob_3_Sleep', 
                                    'State_Prob_4': 'State_Prob_4_Sleep'})

df_sleep = df_sleep[['p_id','timepoint', 'IDS_TOTAL', 'HMM_State_Sleep','State_Prob_1_Sleep', 'State_Prob_2_Sleep',
       'State_Prob_3_Sleep','State_Prob_4_Sleep', 'sleep_day',
       'total_sleep_time_mean', 'total_sleep_time_sd', 'awake_pct_mean',
       'sleep_onset_mean', 'sleep_onset_sd', 'sleep_offset_mean',
       'sleep_offset_sd', 'sleep_efficiency_mean', 'sleep_efficiency_sd']]

# ACTIVITY !!!!
df_activity = df_activity.rename(columns={'hmm': 'HMM_State_Act', 
                                    'State_Prob_1': 'State_Prob_1_Act',
                                    'State_Prob_2': 'State_Prob_2_Act',
                                    'State_Prob_3': 'State_Prob_3_Act'})

df_activity = df_activity[['p_id','timepoint','HMM_State_Act', 'State_Prob_1_Act', 'State_Prob_2_Act',
                           'State_Prob_3_Act','act_day',
                           'Sedentary_time_mean',
                           'Light_activity_mean', 'Moderate_activity_mean',
                           'Vigorous_activity_mean', 'Nighttime_activity_mean',
                           'Total_Daily_Calories_mean', 'Sedentary_time_sd', 'Light_activity_sd',
                           'Moderate_activity_sd', 'Vigorous_activity_sd', 'Nighttime_activity_sd',
                           'Total_Daily_Calories_sd']]              

**Sleep + Baseline (predictors) dataset**

Gender: Female = 0; Male =1 
Deciding against using dummy variables & using the probabilites instead -- if I was to use the cluster memberships again MAKE SURE TO change the code to not include NAs as zeros. 

In [ ]:
# Merge sleep & act data 
df_sleep_pre = pd.merge(df_sleep, df_predictors, on = ['p_id', 'timepoint'], how= 'inner')

**Sleep + Activity + Baseline (predictors)**

In [ ]:
df_sleep_act = pd.merge(df_activity, df_sleep, on = ['p_id', 'timepoint'], how= 'inner')
df_sleep_act_pre = pd.merge(df_sleep_act, df_predictors, on = ['p_id', 'timepoint'], how= 'inner')

### The two final samples 
**Sample 1 = sleep only**

**Sample 2 = sleep + activity**

In [ ]:
sample_1 = df_sleep_pre
sample_2 = df_sleep_act_pre

### Remove where timepoint == 0

In [ ]:
# Remove rows where 'Score' equals 0
sample_1 = sample_1[sample_1['timepoint'] != 0]
sample_2 = sample_2[sample_2['timepoint'] != 0]

**Exploring missingness**

In [ ]:
# Set the display option to show all rows
pd.set_option('display.max_rows', None)
missing_values_df = sample_1.isna().sum().reset_index()  #<--- CHANGE HERE
non_missing_values_df = sample_1.notna().sum().reset_index(drop=True) #<--- CHANGE HERE
na_df = missing_values_df
na_df.columns = ['Variable', 'Missing Values']
na_df['Non-missing Values'] = non_missing_values_df
print(na_df)
pd.reset_option('display.max_rows')

# Random Forest - Nested CV 

Here we have an outer and an inner loop. The outer loop is used to evaluate the model & the inner loop to tune the hyperparameters. 
1. Outer Loop: Use GroupKFold for cross-validation (same as above)
2. Inner Loop: Use GridSearchCV within each of these folds from the outer loop to tune the hyperparameters

#### Creating feature set dictionary 

In [ ]:
chara = ['Baseline_IDS_TOTAL',
            'age_all', 'gender_num', 'mh_family_depr___0', 'mh_family_depr___1', 'mh_family_depr___2', 'LIFETIME_TRAUMA',
            'Mental_comorbidity', 'Physical_comorbidity'
           ]

chara_wo_baselineids = [
            'age_all', 'gender_num', 'mh_family_depr___0', 'mh_family_depr___1', 'mh_family_depr___2', 'LIFETIME_TRAUMA',
            'Mental_comorbidity', 'Physical_comorbidity'
           ]

sleep_cluster = ['State_Prob_1_Sleep', 'State_Prob_2_Sleep', 'State_Prob_3_Sleep', 'State_Prob_4_Sleep']

sleep_act_cluster = ['State_Prob_1_Sleep', 'State_Prob_2_Sleep', 'State_Prob_3_Sleep', 'State_Prob_4_Sleep', 
                      'State_Prob_1_Act', 'State_Prob_2_Act', 'State_Prob_3_Act']

sleep_rmt = ['total_sleep_time_mean', 'total_sleep_time_sd', 'awake_pct_mean',
       'sleep_onset_mean', 'sleep_onset_sd', 'sleep_offset_mean',
       'sleep_offset_sd', 'sleep_efficiency_mean', 'sleep_efficiency_sd']

sleep_act_rmt = ['total_sleep_time_mean', 'total_sleep_time_sd', 'awake_pct_mean',
       'sleep_onset_mean', 'sleep_onset_sd', 'sleep_offset_mean',
       'sleep_offset_sd', 'sleep_efficiency_mean', 'sleep_efficiency_sd',
                 'Sedentary_time_mean', 'Light_activity_mean', 'Moderate_activity_mean', 'Vigorous_activity_mean', 
       'Nighttime_activity_mean', 'Total_Daily_Calories_mean', 'Sedentary_time_sd', 'Light_activity_sd', 
       'Moderate_activity_sd', 'Vigorous_activity_sd', 'Nighttime_activity_sd', 'Total_Daily_Calories_sd']

#Then define a dictionary with combinations.

# to use with df_sleep_pre (SAMPLE 1 - Sleep only)
S1_feature_dict = {'chara_only_S1': chara,
           'chara_and_sleepcluster': chara + sleep_cluster, 
           'chara_and_sleepRMT': chara + sleep_rmt, 
           'chara_and_sleepcluster_sleepRMT': chara + sleep_cluster + sleep_rmt, 

           'chara_wo_b_ids_S1': chara_wo_baselineids, 
           'chara_wo_b_ids_and_sleepcluster': chara_wo_baselineids + sleep_cluster, 
           'chara_wo_b_ids_and_sleepRMT': chara_wo_baselineids + sleep_rmt, 
           'chara_wo_b_ids_and_sleepcluster_and_sleepRMT': chara_wo_baselineids + sleep_cluster + sleep_rmt,
}

# to use with df_sleep_act_pre (SAMPLE 2 - Sleep + Activity)
S2_feature_dict = {'chara_only_S2': chara,
           'chara_and_allCluster': chara + sleep_act_cluster, 
           'chara_and_allRMT': chara + sleep_act_rmt, 
           'chara_and_rmt_and_allClusters': chara + sleep_act_cluster + sleep_act_rmt,
            
           'chara_wo_b_ids_S2': chara_wo_baselineids,
           'chara_wo_b_ids_and_allCluster': chara_wo_baselineids + sleep_act_cluster, 
           'chara_wo_b_ids_and_allRMT': chara_wo_baselineids + sleep_act_rmt, 
           'chara_wo_b_ids_and_allClusters_and_allRMT': chara_wo_baselineids + sleep_act_cluster + sleep_act_rmt,
           }

#### Selecting the relevant feature dictionary & DATASET 

In [ ]:
# sample_1 or sample_2
df = sample_2                                       ####  <- CHANGE SAMPLE HERE 

# Define pipeline and parameter grid 
pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), 
    ('scaler', StandardScaler()),
    ('randomforest', RandomForestRegressor())
])

# Define hyperparameters to tune
rf_param_grid = {
    'randomforest__n_estimators': [10, 25, 50, 100, 250, 500],
    'randomforest__max_depth': [5, 10, 25, 50, 100, 250, None],
    'randomforest__min_samples_split': [10, 25, 50]
}

outer = GroupKFold(n_splits=5)
inner = GroupKFold(n_splits=5)

# Dictionary to store results for each feature set
results_dict = {}
best_params = {}

# Iterate through each feature set in the features dictionary
for features, selected_features in S2_feature_dict.items():     ####  <- CHANGE DICT NAME HERE 
    
    # Prepare the independent variables (IVs) and target variable
    X = df[selected_features]
    y = df['IDS_TOTAL']
    groups = df['p_id']
        
    # Set up inner grid as a gridsearchCV object
    grid = GridSearchCV(pipeline,
                        rf_param_grid,
                        verbose = 1,
                        scoring = 'neg_mean_squared_error',
                        cv=inner)
    
    results = cross_validate(grid, X, y, 
                             cv=outer,
                             groups=groups,
                             # don't understand this row
                             params={'groups': groups}, 
                             scoring=scorers
                             #n_jobs=-1
                            )
    # Store the results in the dictionary
    results_dict[features] = results      
    grid.fit(X, y, groups=groups)  # Fit the grid to get best parameters
    best_params[features] = grid.best_params_                
    
    print(f"Results for {features} stored.")
    print("----------------------------------------------" )

In [ ]:
print(len(X), len(y), len(groups))
group_id = groups[:len(X)]  

In [ ]:
# Initialize an empty list to hold the results
final_results_list = []

# Loop through results_dict and compute the required metrics
for features, results in results_dict.items():
    final_results_list.append({
        'feature_set': features,
        'rmse_mean': results['test_rmse'].mean(),
        'rmse_sd': results['test_rmse'].std(),
        'r2_mean': results['test_r2'].mean(), 
        'r2_sd': results['test_r2'].std(), 
        'mae_mean': results['test_mae'].mean(),
        'mae_sd': results['test_mae'].std()
    })

final_results = pd.DataFrame(final_results_list).set_index('feature_set')
final_results = final_results.reset_index()
display(final_results)

In [ ]:
# Initialize an empty list to hold the results
final_results_list = []

# Loop through results_dict and compute the required metrics
for features, results in results_dict.items():
    final_results_list.append({
        'feature_set': features,
        'rmse': f"{results['test_rmse'].mean():.1f} ({results['test_rmse'].std():.1f})",
        'r2': f"{results['test_r2'].mean():.1f} ({results['test_r2'].std():.1f})",
        'mae': f"{results['test_mae'].mean():.1f} ({results['test_mae'].std():.1f})"
    })

row_order = ["chara_only_S2", "chara_wo_b_ids_S2", 
             "chara_and_allCluster", "chara_wo_b_ids_and_allCluster", 
             "chara_and_allRMT", "chara_wo_b_ids_and_allRMT", 
             "chara_and_rmt_and_allClusters", "chara_wo_b_ids_and_allClusters_and_allRMT"]

final_results = pd.DataFrame(final_results_list).set_index('feature_set')
final_results = final_results.reset_index()

# Reorder rows 
final_results["feature_set"] = pd.Categorical(final_results["feature_set"], categories=row_order, ordered=True)
final_results = final_results.sort_values("feature_set").reset_index(drop=True)
display(final_results)

In [ ]:
final_results.set_index("feature_set", inplace=True)
df_transposed = final_results.T

display(df_transposed)
df_transposed = df_transposed.reset_index()

# Part 2: Feature Importance
**Refit to entire dataset (include inner loop - no outer)**


In [ ]:
# Select the key for the feature set you want to use
selected_feature_key = 'chara_and_rmt_and_allClusters'          # <----   CHANGE HERE
selected_features = S2_feature_dict[selected_feature_key]         

# Prepare the independent variables (IVs) for the regression model
X = sample_2[selected_features]    # <----   CHANGE HERE
y = sample_2['IDS_TOTAL']          # <----   CHANGE HERE
groups = sample_2['p_id']          # <----   CHANGE HERE

# Use GroupKFold for inner cross-validation
inner = GroupKFold(n_splits=5)

# Perform grid search with cross-validation
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=rf_param_grid,
    scoring = 'neg_mean_squared_error',
    cv=inner,
    verbose=1
)
 
# Refit the grid search on the entire dataset
grid_search.fit(X, y, groups=groups)

# Get the best model and its coefficients
best_model = grid_search.best_estimator_
print(best_model)

# Access the RandomForest model inside the pipeline
randomforest_model = best_model.named_steps['randomforest'] 
feature_importance = pd.Series(randomforest_model.feature_importances_, index=X.columns)

# Sort and display feature importance
feature_importance_withbaseline = feature_importance.abs().sort_values(ascending=False)
print(feature_importance_withbaseline)

In [ ]:
# Select the key for the feature set you want to use
selected_feature_key = 'chara_wo_b_ids_and_allClusters_and_allRMT'          # <----   CHANGE HERE
selected_features = S2_feature_dict[selected_feature_key]         

# Prepare the independent variables (IVs) for the regression model
X = sample_2[selected_features]    # <----   CHANGE HERE
y = sample_2['IDS_TOTAL']          # <----   CHANGE HERE
groups = sample_2['p_id']          # <----   CHANGE HERE

# Use GroupKFold for inner cross-validation
inner = GroupKFold(n_splits=5)

# Perform grid search with cross-validation
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=rf_param_grid,
    scoring = 'neg_mean_squared_error',
    cv=inner,
    verbose=1
)
 
# Refit the grid search on the entire dataset
grid_search.fit(X, y, groups=groups)

# Get the best model and its coefficients
best_model = grid_search.best_estimator_
print(best_model)

# Access the RandomForest model inside the pipeline
randomforest_model = best_model.named_steps['randomforest'] 
feature_importance = pd.Series(randomforest_model.feature_importances_, index=X.columns)

# Sort and display feature importance
feature_importance_NObaseline = feature_importance.abs().sort_values(ascending=False)
print(feature_importance_NObaseline)